In [1]:
import numpy as np  # import auxiliary library, typical idiom
import pandas as pd  # import the Pandas library, typical idiom
from numba import jit
import matplotlib
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras

In [2]:
df = pd.read_csv('export.csv')

In [3]:
df_next_event = df.copy()

#A function for determining the true next event for each event
@jit(parallel = True)
def calculator_nb(case, event):
    res = np.empty(len(case), dtype=object)
    idx = 0
    for _ in case:
        if (idx+1 >= len(case)):
            break
       
        if (case[idx + 1] == case[idx]):
            res[idx] = event[idx + 1]

        idx+=1
    return res

df_next_event['next_event'] = calculator_nb(df_next_event['case'].values, df_next_event['event'].values)

C:\Users\mikok\AppData\Local\Temp\ipykernel_19360\3644305937.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_19360\3644305937.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Temp\ipykernel_19360\3644305937.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "calculator_nb" failed type inference due to: Untyped global name 'object': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_19360\3644305937.py", line 6:
<source missing, REPL/exec in use?>

  @jit(parallel = True)
C:\Users\mikok\AppData\Local\Programs\Python\Python310\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "calculator_n

In [4]:
df = df_next_event

In [9]:
goodshit = ['case','event','number_parcels','payment_actual0','area','cross_compliance','penalty_amount0','next_event']
df_reduced = df.copy()

for col in df.columns:
    if (goodshit.count(col) == 0):
        df_reduced = df_reduced.drop(col, 1)

C:\Users\mikok\AppData\Local\Temp\ipykernel_19360\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_19360\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_19360\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_19360\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_1936

C:\Users\mikok\AppData\Local\Temp\ipykernel_19360\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_19360\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_19360\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_19360\3848624921.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_reduced = df_reduced.drop(col, 1)
C:\Users\mikok\AppData\Local\Temp\ipykernel_1936

In [10]:
def normalize(col_name):
    col_as_array = df_reduced[col_name].to_numpy()
    col_as_array = np.where(col_as_array == 0, 0.5, col_as_array)
    col_as_array_norm = np.log10(col_as_array)
    mean = col_as_array_norm.mean()
    stdev = col_as_array_norm.std()
    epsilon = 0.01
    return (col_as_array_norm - mean) / (stdev + epsilon)

In [11]:
def col_to_millis(col_name):
    col_as_array = df_reduced[col_name].to_numpy()
    col_as_array[:] = [elem[:19] for elem in col_as_array]
    
    result = []
    for date in col_as_array:
        utctime = datetime.strptime(date, "%Y/%m/%d %H:%M:%S")
        result.append((utctime - datetime(1970, 1, 1)) // timedelta(milliseconds=1))
    
    return result

In [12]:
df_reduced = df_reduced.fillna(value="None")

In [13]:
df_reduced, validate, test = np.split(df_reduced.sample(frac=1), [int(.6*len(df_reduced)), int(.8*len(df_reduced))])

In [14]:
event = df_reduced['event'].to_numpy()
next_event = df_reduced['next_event'].to_numpy()

In [15]:
encoded_events = [one_hot(d, 43) for d in event]
encoded_events = pad_sequences(encoded_events, maxlen = 3, padding = 'post')

In [16]:
number_parcels = normalize('number_parcels')
payment_actual0 = normalize('payment_actual0')
area = normalize('area')
cross_compliance = normalize('cross_compliance')
penalty_amount0 = normalize('penalty_amount0')

In [17]:
training = []
for i in range(len(encoded_events)):
    current = encoded_events[i]
    current = np.append(current,number_parcels[i])
    current = np.append(current,payment_actual0[i])
    current = np.append(current,area[i])
    current = np.append(current,cross_compliance[i])
    current = np.append(current,penalty_amount0[i])
    training.append(current)

In [18]:
next_event = np.reshape(next_event, (-1,1))

In [19]:
labels = []
for i in range(len(next_event)):
    current = next_event[i]
    current = np.append(current,['','','','','','',''])
    labels.append(current)

In [20]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(8,)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(43, activation='softmax')
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [23]:
model.fit(training,labels,epochs=1,verbose=2)

KeyboardInterrupt: 